In [ ]:
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev
!git clone -b v2.8 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib
!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.8.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

In [ ]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *
import random
import math

@human_readable_agent
def agent(obs):
    if Action.Dribble in obs['sticky_actions']:
        return Action.ReleaseDribble
    #Action.ReleaseDirection
    #Action.ReleaseSprint
    
    if GameMode.KickOff == obs['game_mode']:
        return Action.ShortPass
    if GameMode.Penalty == obs['game_mode']:
        return Action.Shot
    if GameMode.GoalKick == obs['game_mode']:
        return Action.LongPass
    if GameMode.Corner == obs['game_mode']:
        return Action.LongPass
    if GameMode.FreeKick == obs['game_mode']:
        return Action.Shot
    if GameMode.ThrowIn == obs['game_mode']:
        return Action.ShortPass
    if GameMode.Normal == obs['game_mode']:
        if Action.Sprint not in obs['sticky_actions']:
            return Action.Sprint
        controlled_player_pos = obs['left_team'][obs['active']]
        if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
            player_x, player_y = controlled_player_pos[0], controlled_player_pos[1]
            if controlled_player_pos[0] > 0.5:
                goalkeeper_x = obs["right_team"][0][0] + obs["right_team_direction"][0][0] * 13
                goalkeeper_y = obs["right_team"][0][1] + obs["right_team_direction"][0][1] * 13
                x1, x2, y1, y2 = player_x, player_y, goalkeeper_x, goalkeeper_y
                if math.sqrt((x1 - x2) ** 2 + (y1 * 2.38 - y2 * 2.38) ** 2) < 0.3:
                    return Action.Shot
                else:
                    dribbled = False
                    for i in range(1, len(obs["right_team"])):
                        # if opponent is ahead of player
                        if obs["right_team"][i][0] > (player_x - 0.02): 
                            #distance_to_opponent
                            x1, x2, y1, y2 = player_x, player_y, obs["right_team"][i][0], obs["right_team"][i][1]
                            if math.sqrt((x1 - x2) ** 2 + (y1 * 2.38 - y2 * 2.38) ** 2) < 0.03:
                                dribbled = True
                                return Action.Dribble
                                break
                    if dribbled == False:
                        return random.choice([Action.Right, Action.Right, Action.Shot])
            return random.choice([Action.BottomRight, Action.TopRight, Action.Right, Action.HighPass, Action.LongPass, Action.HighPass, Action.LongPass, Action.ShortPass])
        else:
            if obs['ball'][0] > controlled_player_pos[0] + 0.05:
                return Action.Right
            if obs['ball'][0] < controlled_player_pos[0] - 0.05:
                return Action.Left
            if obs['ball'][1] > controlled_player_pos[1] + 0.05:
                return Action.Bottom
            if obs['ball'][1] < controlled_player_pos[1] - 0.05:
                return Action.Top
            return Action.Slide

In [ ]:
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
output = env.run(["/kaggle/working/submission.py", "run_right"])[-1]
print('Left player: reward = %s, status = %s, info = %s' % (output[0]['reward'], output[0]['status'], output[0]['info']))
print('Right player: reward = %s, status = %s, info = %s' % (output[1]['reward'], output[1]['status'], output[1]['info']))
env.render(mode="human", width=800, height=600)